In [45]:
import pandas as pd
import pickle
from bs4 import BeautifulSoup as bs
import time, requests, os, re, random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

In [ ]:
trails_url = 'https://www.alltrails.com/us/washington'
num_trails = 1000

In [46]:
chromedriver = '/Applications/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

option = webdriver.ChromeOptions()
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument("--incognito")
option.add_argument("--disable-plugins-discovery")
option.add_argument("--start-maximized")
option.add_argument('--single-process')
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option("excludeSwitches", ["enable-automation"])

option.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")
browser = webdriver.Chrome(chromedriver, options=option)

/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/853799868.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chromedriver, options=option)


In [47]:
browser.get('https://www.alltrails.com/login')
time.sleep(1)

In [49]:
email = browser.find_element_by_name('userEmail')
email.send_keys('nat.goodby@gmail.com')
time.sleep(1)

pwd = browser.find_element_by_name('userPassword')
pwd.send_keys('9D2J2c7wrg^j')
pwd.submit
time.sleep(1)

browser.find_element_by_xpath("//input[@type='submit']").click()

/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2238880594.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email = browser.find_element_by_name('userEmail')
/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2238880594.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pwd = browser.find_element_by_name('userPassword')
/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2238880594.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath("//input[@type='submit']").click()


In [ ]:
def get_trail_urls(browser, trails_url, num_trails):
    time.sleep(1)
    browser.get(trails_url)
    
    count = num_trails / 10
    while count > 1:
        button = browser.find_element_by_xpath("//button[@title='Show more trails']")
        time.sleep(1)
        try:
            button.click()
            count -= 1
            time.sleep(1)
            browser.execute_script("window.stop();")
            
        except NoSuchElementException:
            time.sleep(1)
            browser.execute_script("window.stop();")
    
    url_list = []
    urls = browser.find_elements_by_xpath("//*[@class='xlate-none styles-module__link___EEZXn']")
    for url in urls:
        next_url = url.get_attribute("href")
        url_list.append(next_url)
    return url_list

In [ ]:
url_list = get_trail_urls(browser, trails_url, num_trails)

In [50]:
#Define functions to scape desired info from each webpage

def get_trail_name(soup):
    try:
        trail_name = soup.find(class_ = 'xlate-none styles-module__name___pgMB8').text
    except AttributeError:
        trail_name = None  
    return trail_name

def get_difficulty(soup):
    try:
        difficulty = soup.find(class_ = 'styles-module__info___jMR_5 styles-module__thin___SHeMO').next.text    
    except AttributeError:
        difficulty = None
    return difficulty
    
def get_distance(soup):
    try:
        distance = soup.find_all(class_ = 'styles-module__detailData___fxmwv')[0].text
        re.sub('[^\d\.]','', distance)
    except AttributeError:
        distance = None
    except IndexError:
        distance = None
    return distance

def get_elevation_gain(soup):
    try:
        elevation_gain = soup.find_all(class_ = 'styles-module__detailData___fxmwv')[1].text
        re.sub('[^0-9]','', elevation_gain)
    except AttributeError:
        elevation_gain = None
    except IndexError:
        elevation_gain = None
    return elevation_gain

def get_route_type(soup):
    try:
        route_type = soup.find_all(class_ = 'styles-module__detailData___fxmwv')[2].text
    except AttributeError:
        route_type = None
    except IndexError:
        route_type = None
    return route_type
    
def get_num_completed(soup):
    try:
        num_completed = soup.find_all(class_='MuiTab-wrapper styles-module__tabWrapper___QY_wC')[-1].text
        re.sub('[^0-9]','', num_completed)
    except AttributeError:
        num_completed = None
    except IndexError:
        num_completed = None
    return num_completed

def get_activities(soup):
    try:
        activities = soup.find_all(class_='MuiTab-wrapper styles-module__tabWrapper___QY_wC')[-2].text
        re.sub('[^0-9]','', activities)
    except AttributeError:
        activities = None
    except IndexError:
        activities = None
    return activities

def get_photos(soup):
    try:
        photos = soup.find_all(class_='MuiTab-wrapper styles-module__tabWrapper___QY_wC')[-3].text
        re.sub('[^0-9]','', photos)
    except AttributeError:
        photos = None
    except IndexError:
        photos = None
    return photos

def get_reviews(soup):
    try:
        reviews = soup.find_all(class_='MuiTab-wrapper styles-module__selectedTabWrapper___Rpj4b')[-1].text
        re.sub('[^0-9]','', reviews)
    except AttributeError:
        reviews = None
    except IndexError:
        reviews = None
    return reviews


def get_tags_list(soup):
    tags = soup.find_all(class_='big rounded active')
    list_of_tags = []
    for tag in tags:
        try:
            list_of_tags.append(tag.text)
        except AttributeError:
            continue
    return list_of_tags

In [ ]:
def scrape_data(url_list):
    trail_info = []
    for count, url in enumerate(url_list):
        time.sleep(.5+2*random.random())
    
        try:
            browser.get(url)
            page = browser.page_source
            soup = bs(page)
            
            trail_dict = {'trail_name':get_trail_name(soup),
                      'difficulty':get_difficulty(soup),
                      'distance':get_distance(soup),
                      'elevation_gain':get_elevation_gain(soup),
                      'route_type':get_route_type(soup),
                      'num_completed':get_num_completed(soup),
                      'activities':get_activities(soup),
                      'photos':get_photos(soup),
                      'reviews':get_reviews(soup),
                      'tags':get_tags_list(soup)}
            trail_info.append(trail_dict)
            print(f"Trail number {count+1}: {trail_dict['trail_name']}")
            browser.execute_script("window.stop();")
        
        except NoSuchElementException:
            time.sleep(5)
            browser.execute_script("window.stop();")
            
        except WebDriverException:
            browser.switch_to.default_content()
            time.sleep(5)
            browser.execute_script("window.stop();")
        
        if (count % 25 == 0):
            trail_df = pd.DataFrame(trail_info)
            trail_df.to_pickle('trails_df.pkl')
            time.sleep(10)
            
    trail_df = pd.DataFrame(trail_info)
    trail_df.to_pickle('trails_df.pkl')
    print(f"The dataframe includes data for {len(trail_df)} trails")
    return trail_df

In [ ]:
scrape_data(url_list)

In [ ]:
url = pd.Series(url_list, name = 'url')
name_and_url = pd.concat([trail_df['trail_name'], url], axis = 1)

In [ ]:
missing_urls = name_and_url[name_and_url['trail_name'].isna()]['url'].tolist()

In [ ]:
def add_failed_trails(missing_urls):
    """
    Function to scrape info for trails that failed during first pass because of RECAPTCHA.
    """   
    all_trails = pd.read_pickle('trails_df.pkl')
    trail_info = []
    for count, url in enumerate(missing_urls):
        time.sleep(.5+2*random.random())
    
        try:
            browser.get(url)
            page = browser.page_source
            soup = bs(page)
            
            trail_dict = {'trail_name':get_trail_name(soup),
                      'difficulty':get_difficulty(soup),
                      'distance':get_distance(soup),
                      'elevation_gain':get_elevation_gain(soup),
                      'route_type':get_route_type(soup),
                      'num_completed':get_num_completed(soup),
                      'activities':get_activities(soup),
                      'photos':get_photos(soup),
                      'reviews':get_reviews(soup),
                      'tags':get_tags_list(soup)}
            all_trails = all_trails.append(trail_dict, ignore_index=True)
            print(f"Trail number {count+1}: {trail_dict['trail_name']}")
            name_and_url['trail_name'] = trail_dict['trail_name']
            browser.execute_script("window.stop();")
        
        except NoSuchElementException:
            time.sleep(5)
            browser.execute_script("window.stop();")
            
        except WebDriverException:
            browser.switch_to.default_content()
            time.sleep(5)
            browser.execute_script("window.stop();")
        
        if (count % 10 == 0):
            time.sleep(10)
       
    all_trails.to_pickle('trails_df.pkl')
    print(f"There are now {all_trails['trail_name'].isnull().sum()} trails without info.")
    return all_trails

In [ ]:
add_failed_trails(missing_urls)

In [23]:
url_list = pd.read_pickle('url_list')

In [43]:
extra_urls = url_list.loc[596:599]

In [52]:
all_trails = pd.read_pickle('trails_df.pkl')

for count, url in enumerate(extra_urls):
    time.sleep(.5+2*random.random())
    try:
        browser.get(url)
        page = browser.page_source
        soup = bs(page)
            
        trail_dict = {'trail_name':get_trail_name(soup),
                    'difficulty':get_difficulty(soup),
                    'distance':get_distance(soup),
                    'elevation_gain':get_elevation_gain(soup),
                    'route_type':get_route_type(soup),
                    'num_completed':get_num_completed(soup),
                    'activities':get_activities(soup),
                    'photos':get_photos(soup),
                    'reviews':get_reviews(soup),
                    'tags':get_tags_list(soup)}
        all_trails = all_trails.append(trail_dict, ignore_index=True)
        print(f"Trail number {count+1}: {trail_dict['trail_name']}")
        browser.execute_script("window.stop();")
        
    except NoSuchElementException:
        time.sleep(5)
        browser.execute_script("window.stop();")
            
    except WebDriverException:
        browser.switch_to.default_content()
        time.sleep(5)
        browser.execute_script("window.stop();")
       
all_trails.to_pickle('all_trails_complete.pkl')

/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2087163418.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_trails = all_trails.append(trail_dict, ignore_index=True)


Trail number 1: Lacamas Lake Heritage Trail


/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2087163418.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_trails = all_trails.append(trail_dict, ignore_index=True)


Trail number 2: Five Mile Drive


/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2087163418.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_trails = all_trails.append(trail_dict, ignore_index=True)


Trail number 3: Lake Lillian Trail
Trail number 4: Little Mountain Park Loop


/var/folders/bh/c8vgcl4518l2jtlcnl312lt00000gn/T/ipykernel_74831/2087163418.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_trails = all_trails.append(trail_dict, ignore_index=True)
